In [1]:
from usefulFns import *

Load Data Here

In [ ]:
allTrials, header = loadMatFilePyMat('Meetz', '221010', 'Meetz_221010_MTNC_Spikes.mat')


Pre-process data for assertions and initialize pandas dataframe for unique stimulus configurations



In [ ]:
if not os.path.exists('Normalization'):
    os.makedirs('Normalization')
os.chdir('Normalization/')


## generate list of unique active units
units = activeUnits('spikeData', allTrials)


## list of indices of correctTrials (non-instruct, valid trialCertify)
corrTrials = correctTrialsMTX(allTrials)


## change stimDesc to be list of dictionaries 
for corrTrial in corrTrials:
    currTrial = allTrials[corrTrial]
    nStim = len(currTrial['stimDesc']['data']['listType'])
    currTrial['stimDesc']['data'] = [{k:v[i] for k,v in currTrial['stimDesc']['data'].items()} 
                                    for i in range(nStim)]


## assert: stim sequence list is frozen
seqList = []
for corrTrial in corrTrials:
    currTrial = allTrials[corrTrial]
    stimDesc = currTrial['stimDesc']['data']
    for stim in stimDesc:
        if stim['stimLoc'] == 0 and stim['listType'] == 1:
            if len(seqList) < 49:
                seqList.append(stim['stimIndex'])
                seqArr = np.array(seqList)
                lastIndex = stim['stimIndex']
            else:
                posLastIndex = np.where(seqArr==lastIndex)[0][0]
                if posLastIndex == len(seqArr)-1:
                    if stim['stimIndex'] != seqArr[0]:
                        print('out of sequence')
                    else:
                        lastIndex = stim['stimIndex']
                else:
                    if stim['stimIndex'] != seqArr[posLastIndex+1]:
                        print('out of sequence')
                    else:
                        lastIndex = stim['stimIndex']


## assert: are there correct trials without spikeData
noSpikeData = []
for trialCount, currTrial in enumerate(allTrials):
    trial = currTrial['trial']['data']
    extendedEOT = currTrial['extendedEOT']['data']
    if extendedEOT == 0 and trial['instructTrial'] != 1:
        if 'spikeData' not in currTrial:
            noSpikeData.append(trialCount)


## assert: frame consistency during stimlus duration
frameRateHz = header['frameRateHz']['data']
stimDurFrame = []
for corrTrial in corrTrials:
    currTrial = allTrials[corrTrial]
    stimDesc = currTrial['stimDesc']['data']
    for stim in stimDesc:
        if stim['stimLoc'] == 0:
            frameDiff = stim['stimOffFrame'] - stim['stimOnFrame']
            stimDurFrame.append(frameDiff)
if len(set(stimDurFrame)) != 1:
    print('stimulus frame duration not consistent for mapping stimuli')
else: 
    trueStimDurMS = np.int32(np.around(1000/frameRateHz * stimDurFrame[0]))


# generates a dictionary, numpy array, and Pandas Dataframe of stim Index 
# and corresponding directions/contrasts
stimIndexDict = {}
for corrTrial in corrTrials:
    currTrial = allTrials[corrTrial]
    stimDesc = currTrial['stimDesc']['data']
    for stim in stimDesc:
        if stim['stimLoc'] != 2:
            index = stim['stimIndex']
            if index not in stimIndexDict:
                stimIndexDict[index] = {}
                if stim['stimLoc'] not in stimIndexDict[index]:
                    stimIndexDict[index][stim['stimLoc']] = \
                    {'direction': stim['directionDeg'],
                        'contrast': np.around(stim['contrast'],2)}
            else:
                if stim['stimLoc'] not in stimIndexDict[index]:
                    stimIndexDict[index][stim['stimLoc']] = \
                    {'direction': stim['directionDeg'],
                        'contrast': np.around(stim['contrast'],2)}
stimIndexArray = np.zeros((49,4))
for i in range(len(stimIndexDict)):
    stimIndexArray[i][0] = stimIndexDict[i][0]['direction']
    stimIndexArray[i][1] = stimIndexDict[i][0]['contrast']
    stimIndexArray[i][2] = stimIndexDict[i][1]['direction']
    stimIndexArray[i][3] = stimIndexDict[i][1]['contrast']
stimIndexDF = pd.DataFrame(stimIndexArray, columns=['loc0 Direction', 'loc0 Contrast',
                                            'loc1 Direction', 'loc1 Contrast'])



Initialize lists/dataframes/arrays for counting spike counts during stim presentations


In [ ]:
#initialize lists/arrays/dataframes for counting spikeCounts and for analysis
blocksDone = allTrials[corrTrials[-2]]['blockStatus']['data']['blocksDone'] 
highContrast, zeroContrast = max(stimIndexDF['loc0 Contrast'].unique()), \
                             min(stimIndexDF['loc0 Contrast'].unique())
zeroDir = 0
dirArray = np.array([0,60,120,180,240,300])
spikeCountMat = np.zeros((len(units),blocksDone+1,49))
spikeCountLong = []
sponSpikeCountLong = []
histPrePostMS = 100 #100ms window pre/post stimlus on/off
sponWindowMS = 50 #50ms window before stimulus onset
spikeHists = np.zeros((len(units),49, trueStimDurMS+(2*histPrePostMS+1)))
stimIndexCount = np.zeros(49) 

Put spike counts into dataframe/array for each stimulus and for each unit 

In [1]:
# insert spike counts into matrix of unique stimulus sets
for corrTrial in corrTrials:
    currTrial = allTrials[corrTrial]
    if 'spikeData' in currTrial:
        stimDesc = currTrial['stimDesc']['data']
        stim1TimeS = currTrial['taskEvents']['stimulusOn']['time'][0]
        fixateTimeS = currTrial['taskEvents']['fixate']['time']
        for stim in stimDesc:
            if stim['stimLoc'] == 0 and stim['listType'] == 1:
                stimOnTimeS = ((1000/frameRateHz * stim['stimOnFrame'])
                                /1000) + stim1TimeS
                stimOffTimeS = ((1000/frameRateHz * stim['stimOffFrame'])
                                /1000) + stim1TimeS
                stimIndex = np.int32(stim['stimIndex'])
                stCount = int(stimIndexCount[stimIndex])
                stimIndexCount[stimIndex] += 1
                for unitCount, unit in enumerate(units):
                    if unit in currTrial['spikeData']['unit']:
                        unitIndex = np.where(currTrial['spikeData']['unit'] == unit)[0]
                        #added 50ms onset latency for spike counts (100 for offset)
                        unitTimeStamps = currTrial['spikeData']['timeStamp'][unitIndex]
                        stimSpikes = np.where((unitTimeStamps >= (stimOnTimeS+0.05)) & 
                                    (unitTimeStamps <= (stimOffTimeS+0.05)))[0]
                        spikeCountMat[unitCount][stCount][stimIndex] \
                        = len(stimSpikes)
                        spikeCountLong.append([unit, stimIndex, stimIndexCount[stimIndex], len(stimSpikes)])

                        #Spontaneous Spikes
                        sponSpikes = np.where((unitTimeStamps >= (stimOnTimeS-(sponWindowMS/1000))) 
                                            & (unitTimeStamps <= stimOnTimeS))[0]
                        sponSpikeCountLong.append([unit,len(sponSpikes)])

                        #PSTHs
                        stimOnPreSNEV = stimOnTimeS - (histPrePostMS/1000)
                        stimOffPostSNEV = stimOffTimeS + (histPrePostMS/1000)
                        histStimSpikes = unitTimeStamps[((unitTimeStamps >= stimOnPreSNEV)\
                                    & (unitTimeStamps <= stimOffPostSNEV))] - stimOnPreSNEV
                        histStimSpikes = np.int32(histStimSpikes*1000)
                        # spikeHists[unitCount, stimIndex, histStimSpikes] += 1
                        spikeHists[unitCount][stimIndex][histStimSpikes] += 1

NameError: name 'corrTrials' is not defined